# Filtering a Time Series Dataframe with a Range Dataframe
#### Using Polars and Pandas

If you have a large time series that needs to be filtered to valid ranges,
   you have a few options available to you. For example, consider two datasets:
   
   1. A time series dataset which holds timestamps of events sorted in order
   2. A range dataset which holds start/end times of valid events sorted in order

You need to filter the time series dataset to only the valid ranges within the range dataset.

`join_asof` / `merge_asof` will be our tools of choice to solve this problem.

For this example, we'll merge approximately **63M rows with 10,000 rows**.

# Create Sample Data
### Time Series Dataframe

In [20]:
import polars as pl
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

np.random.seed(42)

end   = datetime.now().replace(microsecond=0)
start = end + relativedelta(years=-2)

df_timeseries = pl.DataFrame(
    {
        'timestamp': pl.datetime_range(start, end, interval='1s', eager=True)
    }
)

In [21]:
df_timeseries

timestamp
datetime[μs]
2023-04-25 10:22:14
2023-04-25 10:22:15
2023-04-25 10:22:16
2023-04-25 10:22:17
2023-04-25 10:22:18
…
2025-04-25 10:22:10
2025-04-25 10:22:11
2025-04-25 10:22:12


# Create Sample Data
### Range Dataframe

Every 2 hours, create an interval up to 30 minutes long.

In [22]:
np.random.seed(42)

start_times = [start.replace(minute=0, second=0, microsecond=0) + timedelta(hours = i*2) for i in range(10000)]
end_times   = [start + timedelta(seconds=np.random.randint(0, 1801)) for start in start_times]

df_ranges = pl.DataFrame(
    {
        "start": start_times,
        "end": end_times,
    }
)

Confirm that the values are as expected

In [23]:
df_ranges.with_columns(
    dif = pl.col('end') - pl.col('start')
)

start,end,dif
datetime[μs],datetime[μs],duration[μs]
2023-04-25 10:00:00,2023-04-25 10:18:46,18m 46s
2023-04-25 12:00:00,2023-04-25 12:24:19,24m 19s
2023-04-25 14:00:00,2023-04-25 14:14:20,14m 20s
2023-04-25 16:00:00,2023-04-25 16:21:34,21m 34s
2023-04-25 18:00:00,2023-04-25 18:18:50,18m 50s
…,…,…
2025-08-05 08:00:00,2025-08-05 08:22:02,22m 2s
2025-08-05 10:00:00,2025-08-05 10:25:06,25m 6s
2025-08-05 12:00:00,2025-08-05 12:06:36,6m 36s


# Filter Dataframe with Polars

`join_asof` makes easy work of this. 

First we'll join by the start timestamp to get the nearest start that is ≤ `timestamp`. Then we'll only keep timestamps that actually fall within the matched `[start, end]` interval. `join_asof` only guarantees `timestamp ≥ start`, while the final filtering step guarantees that `timestamp ≤ end`.

In [24]:
df_filtered = (
    df_timeseries.join_asof(
        df_ranges,
        left_on  = 'timestamp',
        right_on = 'start',
        strategy = 'backward'
    )
    .filter(
        pl.col('timestamp') <= pl.col('end')
    )
)

In [25]:
df_filtered

timestamp,start,end
datetime[μs],datetime[μs],datetime[μs]
2023-04-25 12:00:00,2023-04-25 12:00:00,2023-04-25 12:24:19
2023-04-25 12:00:01,2023-04-25 12:00:00,2023-04-25 12:24:19
2023-04-25 12:00:02,2023-04-25 12:00:00,2023-04-25 12:24:19
2023-04-25 12:00:03,2023-04-25 12:00:00,2023-04-25 12:24:19
2023-04-25 12:00:04,2023-04-25 12:00:00,2023-04-25 12:24:19
…,…,…
2025-04-25 10:10:42,2025-04-25 10:00:00,2025-04-25 10:10:46
2025-04-25 10:10:43,2025-04-25 10:00:00,2025-04-25 10:10:46
2025-04-25 10:10:44,2025-04-25 10:00:00,2025-04-25 10:10:46


# Filter Dataframe with Pandas

Let's take a look at how you do this in Pandas. Note that the performance is significantly slower.

In [26]:
df_timeseries_pd = df_timeseries.to_pandas()
df_ranges_pd     = df_ranges.to_pandas()

In [27]:
df_filtered_pd = (
    pd.merge_asof(
        df_timeseries_pd,
        df_ranges_pd,
        left_on  = 'timestamp',
        right_on = 'start',
        direction= 'backward'
    )
    .query('timestamp <= end')
)

# Confirm
### Do they both produce the same dataframe?

In [28]:
print(f'Are the two methods equal?\n{df_filtered.equals(pl.DataFrame(df_filtered_pd))}')

Are the two methods equal?
True
